In [2]:
import ssl
print(ssl.OPENSSL_VERSION)

OpenSSL 1.1.1s  1 Nov 2022


In [3]:
!pip install --upgrade pip

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping


In [4]:
pip install stardist

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')


There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.


Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [4]:
from __future__ import print_function, unicode_literals, absolute_import, division
import sys
import numpy as np
import matplotlib
#matplotlib.rcParams["image.interpolation"] = None
import matplotlib.pyplot as plt


from glob import glob
from tqdm import tqdm
from tifffile import imread
from csbdeep.utils import Path, normalize

from stardist import fill_label_holes, random_label_cmap, calculate_extents, gputools_available
from stardist.matching import matching, matching_dataset
from stardist.models import Config2D, StarDist2D, StarDistData2D

In [5]:
from stardist.models import StarDist2D

# prints a list of available models
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


In [6]:
from stardist.data import test_image_nuclei_2d
from stardist.plot import render_label
from csbdeep.utils import normalize
import matplotlib.pyplot as plt

img = sorted(glob('input_image_small_part.tif'))
img_lab = sorted(glob('input_image_small_part_ground_truth.tif'))



In [126]:
img= list(map(imread,img))
img_lab= list(map(imread,img_lab))

In [128]:
#Random color map labels
np.random.seed(42)
lbl_cmap = random_label_cmap()
     


In [145]:


import numpy as np
def slice_numpy_array(image_array):
    # Get the shape of the original array
    height, width = image_array.shape

    # Ensure that both dimensions are divisible by 16
    new_height = height - (height % 16)
    new_width = width - (width % 16)
    image_array = image_array[:new_height, :new_width]

    # Calculate the dimensions of each slice
    slice_height = new_height // 3
    slice_width = new_width // 3

    # List to store the sliced arrays
    sliced_arrays = []

    # Loop through each row and column to slice the array
    for row in range(3):
        for col in range(3):
            upper = row * slice_height
            left = col * slice_width
            lower = upper + slice_height
            right = left + slice_width

            # Ensure that both dimensions of the subarray are divisible by 16
            subarray = image_array[upper:lower, left:right]
            subarray_height, subarray_width = subarray.shape
            subarray = subarray[:subarray_height - (subarray_height % 16), :subarray_width - (subarray_width % 16)]

            # Append the subarray to the list
            sliced_arrays.append(subarray)

            # Do something with the subarray (in this case, just displaying its shape)
            print(f"Subarray at row {row + 1}, column {col + 1}: {subarray.shape}")

    return sliced_arrays

In [131]:
print(np.shape(img))
print(np.shape(img_lab))

print(np.shape(img_lab[0]))

(1, 528, 777)
(1, 528, 777)
(528, 777)


In [132]:
img = slice_numpy_array(img[0])
img_lab = slice_numpy_array(img_lab[0])

Subarray at row 1, column 1: (176, 256)
Subarray at row 1, column 2: (176, 256)
Subarray at row 1, column 3: (176, 256)
Subarray at row 2, column 1: (176, 256)
Subarray at row 2, column 2: (176, 256)
Subarray at row 2, column 3: (176, 256)
Subarray at row 3, column 1: (176, 256)
Subarray at row 3, column 2: (176, 256)
Subarray at row 3, column 3: (176, 256)
Subarray at row 1, column 1: (176, 256)
Subarray at row 1, column 2: (176, 256)
Subarray at row 1, column 3: (176, 256)
Subarray at row 2, column 1: (176, 256)
Subarray at row 2, column 2: (176, 256)
Subarray at row 2, column 3: (176, 256)
Subarray at row 3, column 1: (176, 256)
Subarray at row 3, column 2: (176, 256)
Subarray at row 3, column 3: (176, 256)


In [133]:
n_channel = 1 if img[0].ndim == 2 else img[0].shape[-1]  #If no third dim. then number of channels = 1. Otherwise get the num channels from the last dim.

In [134]:

#Normalize input images and fill holes in masks
axis_norm = (0,1)   # normalize channels independently
#axis_norm = (0,1,2) # normalize channels jointly
if n_channel > 1:
    print("Normalizing image channels %s." % ('jointly' if axis_norm is None or 2 in axis_norm else 'independently'))
    sys.stdout.flush()

X = [normalize(x,1,99.8,axis=axis_norm) for x in tqdm(img)]
Y = [fill_label_holes(y) for y in tqdm(img_lab)]

100%|██████████| 9/9 [00:00<00:00, 387.84it/s]


In [135]:
#Split to train and val
#You can use any method to split. I am following the method used in StarDist documentation example
#add more data 

#assert len(img) > 1, "not enough training data"
rng = np.random.RandomState(42)
ind = rng.permutation(len(img))
n_val = max(1, int(round(0.15 * len(ind))))
ind_train, ind_val = ind[:-n_val], ind[-n_val:]
X_val, Y_val = [img[i] for i in ind_val]  , [img_lab[i] for i in ind_val]
X_trn, Y_trn = [img[i] for i in ind_train], [img_lab[i] for i in ind_train]
print('number of images: %3d' % len(img))
print('- training:       %3d' % len(X_trn))
print('- validation:     %3d' % len(X_val))
     


number of images:   9
- training:         8
- validation:       1


In [138]:
print(img.shape)

(176, 256)


In [146]:
img = sorted(glob('input_image_small_part.tif'))
img_lab = sorted(glob('input_image_small_part_ground_truth.tif'))
img= list(map(imread,img))
img_lab= list(map(imread,img_lab))
img = slice_numpy_array(img[0])
img_lab = slice_numpy_array(img_lab[0])

Subarray at row 1, column 1: (176, 256)
Subarray at row 1, column 2: (176, 256)
Subarray at row 1, column 3: (176, 256)
Subarray at row 2, column 1: (176, 256)
Subarray at row 2, column 2: (176, 256)
Subarray at row 2, column 3: (176, 256)
Subarray at row 3, column 1: (176, 256)
Subarray at row 3, column 2: (176, 256)
Subarray at row 3, column 3: (176, 256)
Subarray at row 1, column 1: (176, 256)
Subarray at row 1, column 2: (176, 256)
Subarray at row 1, column 3: (176, 256)
Subarray at row 2, column 1: (176, 256)
Subarray at row 2, column 2: (176, 256)
Subarray at row 2, column 3: (176, 256)
Subarray at row 3, column 1: (176, 256)
Subarray at row 3, column 2: (176, 256)
Subarray at row 3, column 3: (176, 256)


In [7]:
# prints a list of available models
# TO DO- peut etre on peut juste faire pareil avec GT???
StarDist2D.from_pretrained()

# creates a pretrained model
model = StarDist2D.from_pretrained('2D_versatile_fluo')

labels, _ = model.predict_instances(normalize(img[0])) #or for loop if many images

plt.subplot(1,2,1)
plt.imshow(img[0], cmap="gray")
plt.axis("off")
plt.title("input image")

plt.subplot(1,2,2)
plt.imshow(render_label(labels, img=img[0]))
plt.axis("off")
plt.title("prediction + input overlay")

There are 4 registered models for 'StarDist2D':

Name                  Alias(es)
────                  ─────────
'2D_versatile_fluo'   'Versatile (fluorescent nuclei)'
'2D_versatile_he'     'Versatile (H&E nuclei)'
'2D_paper_dsb2018'    'DSB 2018 (from StarDist 2D paper)'
'2D_demo'             None
Found model '2D_versatile_fluo' for 'StarDist2D'.
Loading network weights from 'weights_best.h5'.
Loading thresholds from 'thresholds.json'.
Using default values: prob_thresh=0.479071, nms_thresh=0.3.


UFuncTypeError: ufunc 'subtract' did not contain a loop with signature matching types (dtype('<U26'), dtype('<U26')) -> None

In [155]:
# Plot image and label for some images - sanity check
#print(img.shape)
#TO DO- pb here 

def plot_img_label(img, lbl, img_title="image", lbl_title="label", **kwargs):
    fig, (ai,al) = plt.subplots(1,2, figsize=(12,5), gridspec_kw=dict(width_ratios=(1.25,1)))
    im = ai.imshow(img, cmap='gray', clim=(0,1))

    ai.set_title(img_title)
    fig.colorbar(im, ax=ai)
    al.imshow(lbl, cmap=lbl_cmap)
    al.set_title(lbl_title)
    plt.tight_layout()

i = min(9, len(img)-1)
print(i)
print(len(img))
print(len(img_lab))
if i < len(img) and i < len(img_lab):
    img, lbl = img[i], img_lab[i]
    assert img.ndim in (2,3)
    img = img if (img.ndim==2 or img.shape[-1]==3) else img[...,0]
    for index, x in enumerate(img):
        plot_img_label(img[index], img_lab[index])

9
176
9


In [156]:

labels, _ = model.predict_instances(normalize(img[0])) #or for loop if many images

plt.subplot(1,2,1)
plt.imshow(img[0], cmap="gray")
plt.axis("off")
plt.title("input image")

plt.subplot(1,2,2)
plt.imshow(render_label(labels, img=img[0]))
plt.axis("off")
plt.title("prediction + input overlay")

ValueError: axes (YXC) must be of length 1.